# Data Cleaning

This notebook aims to clean up the play-by-play dataset. There are over 200 features in the dataset, but the first step is determining which of these features are established before the play occurs vs. after the play occurs. The next step is dealing with the non-numeric variables in the data. Finally, I take a look at the nulls.

**Please Note: Restarting this notebook will cause errors in the code since all the data is not stored locally.**

## Contents
[Eliminating Post-play Features](#Eliminating-Post-play-Features)<br>
[Non-Numeric Variables](#Non-Numeric-Variables)<br>
[Dealing with Nulls](#Dealing-with-Nulls)

In [1]:
import pandas as pd
import s3fs
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999999)

import warnings
warnings.filterwarnings('ignore')

In [2]:
fs = s3fs.S3FileSystem(anon=False,key='AWS KEY',secret='AWS SECRET KEY')

key = 'nfl_play_by_play_with_weather_2009_2018.csv'
bucket = 'nfl-play-by-play-capstone'

df = pd.read_csv(fs.open('{}/{}'.format(bucket, key),
                         mode='rb'))

### Eliminating Post-play Features

There are more than 250 variables in this dataset, but many of these variables are information gained after the play occurred. For example, incomplete_pass refers to whether or not a pass was completed on that play. If we are predicting play type, we can't use this information because we would not have it prior to the play. I narrowed the dataframe down to features that are considered pre-play using the definitions of variables found [here](https://github.com/maksimhorowitz/nflscrapR/blob/master/R/scrape_play_by_play.R).

In [3]:
df = df[['forecast', 'week', 'wind', 'year','game_id','home_team','away_team','posteam','posteam_type','defteam',
         'side_of_field','yardline_100','game_date','quarter_seconds_remaining','half_seconds_remaining',
         'game_seconds_remaining','game_half','quarter_end','drive','qtr','down','goal_to_go','time','yrdln',
         'ydstogo','desc','play_type','home_timeouts_remaining','away_timeouts_remaining',
         'posteam_timeouts_remaining','defteam_timeouts_remaining','total_home_score','total_away_score',
         'posteam_score','defteam_score','score_differential','wp']]

### Non-Numeric Variables

In this section, I want to look at the non-numeric data. I will need to do one of three things to these variables: ignore them, extract numeric data from strings, or dummy the columns. First, let's look at the non-numeric columns.

In [4]:
df.select_dtypes(include = 'object').columns

Index(['forecast', 'wind', 'home_team', 'away_team', 'posteam', 'posteam_type',
       'defteam', 'side_of_field', 'game_date', 'game_half', 'time', 'yrdln',
       'desc', 'play_type'],
      dtype='object')

In [5]:
df.select_dtypes(include = 'object').head()

,forecast,wind,home_team,away_team,posteam,posteam_type,defteam,side_of_field,game_date,game_half,time,yrdln,desc,play_type
0,67f Mostly Cloudy,NaN,PIT,TEN,PIT,home,TEN,TEN,2009-09-10,Half1,15:00,TEN 30,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff
1,67f Mostly Cloudy,NaN,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,Half1,14:53,PIT 42,(14:53) B.Roethlisberger pass short left to H....,pass
2,67f Mostly Cloudy,NaN,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,Half1,14:16,PIT 47,(14:16) W.Parker right end to PIT 44 for -3 ya...,run
3,67f Mostly Cloudy,NaN,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,Half1,13:35,PIT 44,(13:35) (Shotgun) B.Roethlisberger pass incomp...,pass
4,67f Mostly Cloudy,NaN,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,Half1,13:27,PIT 44,(13:27) (Punt formation) D.Sepulveda punts 54 ...,punt


First, let's look at forecast. I want to separate the temperature into temperature and description.

In [6]:
df['temperature'] = df['forecast'].str.split('f ',expand=True)[0]
df['weather_desc'] = df['forecast'].str.split('f ',expand=True)[1]
df = df.drop(columns='forecast')

Now let's check the values we have for temperature.

In [7]:
df['temperature'].value_counts()

DOME     91161
63       11087
64        8963
70        8827
62        8556
57        8512
54        8495
61        8400
48        8230
52        8165
60        8099
80        7975
71        7966
66        7937
75        7880
73        7826
78        7741
67        7612
68        7379
72        7313
59        7210
69        7022
53        6958
79        6758
58        6585
77        6447
56        6003
46        5701
65        5684
37        5662
51        5475
45        5041
50        4875
41        4785
82        4743
42        4739
55        4701
35        4628
43        4538
44        4437
34        4409
76        4391
81        4343
47        4045
39        3939
84        3920
85        3825
74        3714
38        3690
49        3614
40        3609
83        3361
32        3341
87        3270
30        3024
88        3022
86        2691
89        2359
28        2270
36        2106
26        2105
33        1774
27        1722
25        1531
31        1384
24        1216
22        

Already I can see that Dome will be difficult to replace or impute. I can't be sure with the information I have whether or not the Dome was open or closed during the game. There are also two games that have it seems two temperatures. I am first going update those temperatures. Then, for now I am going to create a binary column for whether or not a game was in a dome and binary columns for ranges of temperatures (cold < 40 degrees, moderate (41,70), hot > 70 degrees). To create these columns, I am going to impute 65 for temperature in Dome stadiums.

In [8]:
df['temperature'].replace('33/51','51',inplace=True)
df['temperature'].replace('39/53','53',inplace=True)

In [9]:
df['dome'] = np.where(df['temperature'] == 'DOME',1,0)

In [10]:
df['temperature'] = df['temperature'].replace('DOME',65).astype(float)

In [11]:
mask = [df['temperature'].between(-100,45), df['temperature'].between(46,70), df['ydstogo'].between(71,150)]
values = ['Cold','Moderate','Hot']
df['temperature_range'] = np.select(mask, values, 0)

In [12]:
df = df.join(pd.get_dummies(df['temperature_range'], prefix='temp_range',drop_first=True))
df = df.drop(columns=['temperature_range','temperature'])

Next, I need to use the weather description. I decided to create a dummy variable for whether or not there is precipitation during the game. If the forecast was a Dome, the weather description was null. I am going to make the assumption that there is no precipitation in a Dome.

In [13]:
df['weather_desc'].value_counts()

Clear                                             70940
Mostly Cloudy                                     66170
Partly Cloudy                                     54546
Fair                                              52063
Overcast                                          44161
A Few Clouds                                      19280
Light Rain                                         7983
NA                                                 2866
 Light Rain                                        2855
Rain                                               2541
Drizzle                                            2527
 Fog/Mist                                          1787
Humid and Mostly Cloudy                            1626
Fog/Mist                                           1393
Overcast and Breezy                                1240
Mostly Cloudy and Breezy                           1208
Humid and Partly Cloudy                            1097
Light Rain Fog/Mist                             

In [14]:
df['precipitation'] = df['weather_desc'].fillna('NA').apply(lambda x: 1 if ('Rain' in x 
                                                                           or 'Snow' in x 
                                                                           or 'Wintry' in x
                                                                           or 'Drizzle' in x
                                                                           or 'Fog' in x
                                                                           or 'Showers' in x
                                                                           or 'Flurries' in x
                                                                           or 'Blizzard' in x
                                                                           or 'Thunder' in x) else 0)
df = df.drop(columns='weather_desc')

Next, wind speed needs to be extracted from the wind string. NFLWeather.com did not start collecting wind consistently until about 2015. I probably won't use this feature since so many values are missing (30%), but I will still extract the number for what it is worth.

In [15]:
df[df['wind'].isnull()]['year'].max()

2014

In [16]:
df['wind'].isnull().mean()

0.3013156430177423

In [17]:
df['wind_speed'] = df['wind'].str.split('m ', expand=True)[0].astype(float)
df = df.drop(columns='wind')

Pos team & def team will be dummied, but the dummying will created more than 60 additional columns. For storage purposes, I will dummy these in the feature engineering notebook. The next variable to address is whether or not the possession team is home.

In [18]:
df['posteam_home'] = df['posteam_type'].apply(lambda x: 1 if x == 'home' else 0)
df = df.drop(columns=['home_team','away_team','posteam_type'])

I am also going to make the game_date a datetime object rather thana a string.

In [19]:
df['game_date'] = pd.to_datetime(df['game_date'])

Next, I will binarize whether or not the team is on their own side.

In [20]:
df['own_side'] = np.where(df['posteam'] == df['side_of_field'],1,0)
df = df.drop(columns=['side_of_field'])

Next, I am going to eliminate plays in overtime and binarize whether or not it is the second half.

In [21]:
df['game_half'].value_counts()

Half2       223879
Half1       222079
Overtime      3252
Name: game_half, dtype: int64

In [22]:
df = df[df['game_half'] != 'Overtime']

In [23]:
df['second_half'] = df['game_half'].apply(lambda x: 1 if x =='Half2' else 0)
df = df.drop(columns='game_half')

Next I am going to drop time since we already have quarter_seconds_left, half_seconds_left, and game_seconds_left.

In [24]:
df = df.drop(columns = ['time'])

Finally, I am going to drop the column yrdln because we already have yardline_100 with essentially the same information.

In [25]:
df = df.drop(columns='yrdln')

In [26]:
df.select_dtypes(include = 'object').head()

,posteam,defteam,desc,play_type
0,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff
1,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,pass
2,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,run
3,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,pass
4,PIT,TEN,(13:27) (Punt formation) D.Sepulveda punts 54 ...,punt


As I mentioned above, posteam and defteam will be dummied in the feature engineering notebook. Play description will remain, but I will not use it in the model. Playtype will be what we are predicting, so no need to change at the moment.

### Dealing with Nulls

In this section, I aim to address any nulls we have in the data. First I need to isolate the plays to those that are effectively a run (run or qb kneel) or effectively a pass (pass or qb spike).

In [27]:
df.head()

,week,year,game_id,posteam,defteam,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,quarter_end,drive,qtr,down,goal_to_go,ydstogo,desc,play_type,home_timeouts_remaining,away_timeouts_remaining,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,wp,dome,temp_range_Cold,temp_range_Moderate,precipitation,wind_speed,posteam_home,own_side,second_half
0,1,2009,2.009091e+09,PIT,TEN,30.0,2009-09-10,900.0,1800.0,3600.0,0.0,1.0,1.0,NaN,0.0,0.0,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff,3.0,3.0,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0,1,0,NaN,1,0,0
1,1,2009,2.009091e+09,PIT,TEN,58.0,2009-09-10,893.0,1793.0,3593.0,0.0,1.0,1.0,1.0,0.0,10.0,(14:53) B.Roethlisberger pass short left to H....,pass,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.546433,0,0,1,0,NaN,1,1,0
2,1,2009,2.009091e+09,PIT,TEN,53.0,2009-09-10,856.0,1756.0,3556.0,0.0,1.0,1.0,2.0,0.0,5.0,(14:16) W.Parker right end to PIT 44 for -3 ya...,run,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.551088,0,0,1,0,NaN,1,1,0
3,1,2009,2.009091e+09,PIT,TEN,56.0,2009-09-10,815.0,1715.0,3515.0,0.0,1.0,1.0,3.0,0.0,8.0,(13:35) (Shotgun) B.Roethlisberger pass incomp...,pass,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.510793,0,0,1,0,NaN,1,1,0
4,1,2009,2.009091e+09,PIT,TEN,56.0,2009-09-10,807.0,1707.0,3507.0,0.0,1.0,1.0,4.0,0.0,8.0,(13:27) (Punt formation) D.Sepulveda punts 54 ...,punt,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.461217,0,0,1,0,NaN,1,1,0


In [28]:
df['play_type'].value_counts()

pass           185349
run            131598
no_play         42026
kickoff         25362
punt            23749
extra_point     10932
field_goal       9615
qb_kneel         3818
qb_spike          681
Name: play_type, dtype: int64

In [29]:
df = df[(df['play_type'] == 'pass') | (df['play_type'] == 'run') | (df['play_type'] == 'qb_kneel')
       | (df['play_type'] == 'qb_spike')]

In [30]:
df['play_type'].value_counts()

pass        185349
run         131598
qb_kneel      3818
qb_spike       681
Name: play_type, dtype: int64

In [31]:
df['effective_run'] = df['play_type'].map({'pass': 0, 'qb_spike':0, 'run':1, 'qb_kneel':1})

In [32]:
df['effective_pass'] = df['play_type'].map({'pass': 1, 'qb_spike':1, 'run':0, 'qb_kneel':0})

The next step is addressing the nulls.

In [33]:
df.isnull().sum()

week                              0
year                              0
game_id                           0
posteam                           0
defteam                           0
yardline_100                      0
game_date                         0
quarter_seconds_remaining         0
half_seconds_remaining           17
game_seconds_remaining           14
quarter_end                       0
drive                             0
qtr                               0
down                            754
goal_to_go                        0
ydstogo                           0
desc                              0
play_type                         0
home_timeouts_remaining           0
away_timeouts_remaining           0
posteam_timeouts_remaining        0
defteam_timeouts_remaining        0
total_home_score                  0
total_away_score                  0
posteam_score                    10
defteam_score                    10
score_differential               10
wp                          

First, we need to impute half_seconds_remaining & game_seconds_remaining. We have enough information given quarter_seconds_remaining and qtr. In order to impute these, I need to reindex the dataframe since we just dropped a bunch of play types.

In [34]:
df.reset_index(drop=True,inplace=True)

Next, I will impute half_seconds_remaining.

In [35]:
for i in df[df['half_seconds_remaining'].isnull()].index:
    seconds = df.iloc[i]['quarter_seconds_remaining'] + 900
    df.loc[[i],['half_seconds_remaining']] = seconds

Next, I will impute game_seconds_remaining.

In [36]:
for i in df[df['game_seconds_remaining'].isnull()].index:
    if (df.iloc[i]['qtr'] == 3):
        seconds = df.iloc[i]['half_seconds_remaining']
        df.loc[[i],['game_seconds_remaining']] = seconds
    elif (df.iloc[i]['qtr'] == 1) | (df.iloc[i]['qtr'] == 2):
        seconds = df.iloc[i]['half_seconds_remaining'] + 1800
        df.loc[[i],['game_seconds_remaining']] = seconds
    else:
        pass

Next, I will take a look at the missing downs.

In [37]:
df[df['down'].isnull()].head()

,week,year,game_id,posteam,defteam,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,quarter_end,drive,qtr,down,goal_to_go,ydstogo,desc,play_type,home_timeouts_remaining,away_timeouts_remaining,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,wp,dome,temp_range_Cold,temp_range_Moderate,precipitation,wind_speed,posteam_home,own_side,second_half,effective_run,effective_pass
985,1,2009,2.009091e+09,DEN,CIN,2.0,2009-09-13,11.0,11.0,11.0,0.0,21.0,4.0,NaN,0.0,0.0,(Pass formation) TWO-POINT CONVERSION ATTEMPT....,pass,2.0,0.0,0.0,2.0,7.0,12.0,12.0,7.0,5.0,0.958721,0,0,0,0,NaN,0,0,1,0,1
1592,1,2009,2.009091e+09,GB,CHI,2.0,2009-09-13,71.0,71.0,71.0,0.0,24.0,4.0,NaN,0.0,0.0,TWO-POINT CONVERSION ATTEMPT. A.Rodgers pass t...,pass,2.0,0.0,2.0,0.0,21.0,15.0,19.0,15.0,4.0,0.932944,0,0,0,0,NaN,1,0,1,0,1
1713,1,2009,2.009091e+09,NE,BUF,2.0,2009-09-14,126.0,126.0,126.0,0.0,16.0,4.0,NaN,0.0,0.0,TWO-POINT CONVERSION ATTEMPT. T.Brady pass to ...,pass,3.0,2.0,3.0,2.0,19.0,23.0,19.0,23.0,-4.0,0.257564,0,0,0,0,NaN,1,0,1,0,1
1717,1,2009,2.009091e+09,NE,BUF,2.0,2009-09-14,50.0,50.0,50.0,0.0,18.0,4.0,NaN,0.0,0.0,TWO-POINT CONVERSION ATTEMPT. T.Brady pass to ...,pass,3.0,2.0,3.0,2.0,25.0,23.0,25.0,23.0,2.0,0.884033,0,0,0,0,NaN,1,0,1,0,1
3115,2,2009,2.009092e+09,TB,BUF,2.0,2009-09-20,327.0,327.0,327.0,0.0,23.0,4.0,NaN,0.0,0.0,(Pass formation) TWO-POINT CONVERSION ATTEMPT....,pass,3.0,1.0,1.0,3.0,29.0,20.0,20.0,29.0,-9.0,0.150597,0,0,0,0,NaN,0,0,1,0,1


It appears that each play with a missing down is a two-point conversion, a botched PAT attempt, or some kickoffs were labeld as qb kneels in the endzone. Let's check.

In [38]:
counter = 0
for row in df['desc']:
    if 'TWO-POINT' in row:
        counter +=1
counter == df[df['down'].isnull()].shape[0] - df[(df['down'].isnull()) & ~(df['desc'].str.contains('TWO-POINT')) &
                                                 df['desc'].str.contains('kicks')].shape[0]

True

In [39]:
df[(df['down'].isnull()) & ~(df['desc'].str.contains('TWO-POINT'))].shape[0]/df.shape[0]

0.00023954256702525464

All downs that are missing are kickoffs or two-point conversions. For the purposes of this project, I am going to drop the kickoffs & two-point conversions (less than 0.02% of plays).

In [40]:
df = df.dropna(subset=['down'])

Next, we need to look posteam_score, defteam_score, and score_differential. The drops above seem to have gotten rid of those.

In [41]:
df[df['posteam_score'].isnull()].shape

(0, 38)

In [42]:
df[df['defteam_score'].isnull()].shape

(0, 38)

In [43]:
df[df['score_differential'].isnull()].shape

(0, 38)

Next, I am going to drop where win probability is null since that calculation was done outside of this project.

In [44]:
df = df.dropna(subset=['wp'])

The only nulls we have left are wind_speed which simply was not collected by the source NFLWeather.com consistently until 2015. For now, we are going to leave the nulls and ignore the variable.

In [45]:
df.isnull().sum().sort_values(ascending=False).head()

wind_speed                96965
effective_pass                0
half_seconds_remaining        0
ydstogo                       0
goal_to_go                    0
dtype: int64

Finally, let's save the clean csv back to S3.

In [46]:
fs = s3fs.S3FileSystem(anon=False,key='AWS KEY',secret='AWS SECRET KEY')

bucket = 'nfl-play-by-play-capstone'

with fs.open(f'{bucket}/nfl_play_by_play_with_weather_2009_2018_clean.csv','w') as f:
    df.to_csv(f)